Main objective --> evaluate if this hacking was made by 2 or 3 hackers.

Steps: 
1-cluster the important information collected by the affected bussines (except the location because it is not useful in this case). 

2- group the information --> first in two clusters 
                         --> second in three clusters

3- measure de Shillouette parameter  --> evaluate accuracy
                                  a) 2 clusters more accurate --> answer 
                                  b) 3 clusters more acccurate --> answer

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://mirrors.sonic.net/apache/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
!tar xzf spark-3.1.2-bin-hadoop3.2.tgz
!pip install -q findspark


import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"


import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()


In [ ]:
df = spark.read.format('csv').options(inferSchema=True, header=True).load('/content/drive/MyDrive/hack_data.csv')

In [ ]:
df.head()

Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, Location='Slovenia', WPM_Typing_Speed=72.37)

In [ ]:
df.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)



In [ ]:
df.describe().show()


+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|summary|Session_Connection_Time| Bytes Transferred|   Kali_Trace_Used|Servers_Corrupted|   Pages_Corrupted|   Location|  WPM_Typing_Speed|
+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|  count|                    334|               334|               334|              334|               334|        334|               334|
|   mean|     30.008982035928145| 607.2452694610777|0.5119760479041916|5.258502994011977|10.838323353293413|       null|57.342395209580864|
| stddev|     14.088200614636158|286.33593163576757|0.5006065264451406| 2.30190693339697|  3.06352633036022|       null| 13.41106336843464|
|    min|                    1.0|              10.0|                 0|              1.0|               6.0|Afghanistan|              40.0|
|    max|           

In [ ]:
df.columns


['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler


In [ ]:
feat_cols = ['Session_Connection_Time', 'Bytes Transferred', 'Kali_Trace_Used',
             'Servers_Corrupted', 'Pages_Corrupted','WPM_Typing_Speed']

In [ ]:
vec_assembler = VectorAssembler(inputCols = feat_cols, outputCol='features')
df

DataFrame[Session_Connection_Time: double, Bytes Transferred: double, Kali_Trace_Used: int, Servers_Corrupted: double, Pages_Corrupted: double, Location: string, WPM_Typing_Speed: double]

In [ ]:
final_data = vec_assembler.transform(df)
final_data.select("features").show(truncate=False)

+--------------------------------+
|features                        |
+--------------------------------+
|[8.0,391.09,1.0,2.96,7.0,72.37] |
|[20.0,720.99,0.0,3.04,9.0,69.08]|
|[31.0,356.32,1.0,3.71,8.0,70.58]|
|[2.0,228.08,1.0,2.48,8.0,70.8]  |
|[20.0,408.5,0.0,3.57,8.0,71.28] |
|[1.0,390.69,1.0,2.79,9.0,71.57] |
|[18.0,342.97,1.0,5.1,7.0,72.32] |
|[22.0,101.61,1.0,3.03,7.0,72.03]|
|[15.0,275.53,1.0,3.53,8.0,70.17]|
|[12.0,424.83,1.0,2.53,8.0,69.99]|
|[15.0,249.09,1.0,3.39,9.0,70.77]|
|[32.0,242.48,0.0,4.24,8.0,67.93]|
|[23.0,514.54,0.0,3.18,8.0,68.56]|
|[9.0,284.77,0.0,3.12,9.0,70.82] |
|[27.0,779.25,1.0,2.37,8.0,72.73]|
|[12.0,307.31,1.0,3.22,7.0,67.95]|
|[21.0,355.94,1.0,2.0,7.0,72.0]  |
|[10.0,372.65,0.0,3.33,7.0,69.19]|
|[20.0,347.23,1.0,2.33,7.0,70.41]|
|[22.0,456.57,0.0,1.52,8.0,69.35]|
+--------------------------------+
only showing top 20 rows



In [ ]:
from pyspark.ml.clustering import KMeans
kmeans = KMeans(k=2, seed=1)
model = kmeans.fit(final_data)

In [ ]:
results = model.transform(final_data)
results.show()

+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+--------------------+----------+
|Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|            Location|WPM_Typing_Speed|            features|prediction|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+--------------------+----------+
|                    8.0|           391.09|              1|             2.96|            7.0|            Slovenia|           72.37|[8.0,391.09,1.0,2...|         1|
|                   20.0|           720.99|              0|             3.04|            9.0|British Virgin Is...|           69.08|[20.0,720.99,0.0,...|         0|
|                   31.0|           356.32|              1|             3.71|            8.0|             Tokelau|           70.58|[31.0,356.32,1.0,...|         1|
|               

In [ ]:
from pyspark.ml.evaluation import ClusteringEvaluator
evaluator = ClusteringEvaluator(metricName='silhouette', distanceMeasure='squaredEuclidean')

silhouette = evaluator.evaluate(results)
print("Silhouette = " + str(silhouette))

Silhouette = 0.8048521975748283


In [ ]:
from pyspark.ml.feature import StandardScaler

In [ ]:
model.clusterCenters()

[array([4.00060976e+01, 8.59157866e+02, 5.36585366e-01, 6.92634146e+00,
        1.32134146e+01, 4.68612195e+01]),
 array([ 20.36470588, 364.22370588,   0.48823529,   3.64952941,
          8.54705882,  67.45364706])]

In [ ]:
from pyspark.ml.clustering import KMeans
kmeans = KMeans(k=3, seed=1)
model = kmeans.fit(final_data)

results = model.transform(final_data)
results.show()

+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+--------------------+----------+
|Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|            Location|WPM_Typing_Speed|            features|prediction|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+--------------------+----------+
|                    8.0|           391.09|              1|             2.96|            7.0|            Slovenia|           72.37|[8.0,391.09,1.0,2...|         1|
|                   20.0|           720.99|              0|             3.04|            9.0|British Virgin Is...|           69.08|[20.0,720.99,0.0,...|         0|
|                   31.0|           356.32|              1|             3.71|            8.0|             Tokelau|           70.58|[31.0,356.32,1.0,...|         1|
|               

In [ ]:
from pyspark.ml.evaluation import ClusteringEvaluator
evaluator = ClusteringEvaluator(metricName='silhouette', distanceMeasure='squaredEuclidean')

silhouette = evaluator.evaluate(results)
print("Silhouette = " + str(silhouette))

Silhouette = 0.6958546319645178


In [ ]:
model.clusterCenters()

[array([2.86210526e+01, 5.91309684e+02, 5.47368421e-01, 5.12989474e+00,
        1.04315789e+01, 5.86424211e+01]),
 array([ 18.27192982, 286.43087719,   0.45614035,   3.16122807,
          8.01754386,  70.10263158]),
 array([4.1768000e+01, 9.1193904e+02, 5.3600000e-01, 7.2689600e+00,
        1.3720000e+01, 4.4717040e+01])]

Silhouette parameter is higher when the model is 2 clusters, so the answer is 2 hackers.

In [ ]:
#scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=False)


In [ ]:
#StandardScaler
#scalerModel = scaler.fit(final_data)

In [ ]:
# Normalize 
#cluster_final_data = scalerModel.transform(final_data)


In [ ]:
#kmeans3 = KMeans(featuresCol='scaledFeatures',k=3)
#kmeans2 = KMeans(featuresCol='scaledFeatures',k=2)

In [ ]:
#model_k3 = kmeans3.fit(cluster_final_data)
#model_k2 = kmeans2.fit(cluster_final_data)

In [ ]:
#model_k3.transform(cluster_final_data).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         2|   88|
|         0|   79|
+----------+-----+



In [ ]:
#model_k2.transform(cluster_final_data).groupBy('prediction').count().show()


+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         0|  167|
+----------+-----+

